## Семинар 13. Гребневая регрессия.

In [ ]:
import numpy as np
import scipy as sp
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import seaborn as sns
sns.set()

In [ ]:
#from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV

### Задача №1

В файле "House_prices_corrected.csv" представлены характеристики различных домов (стоимость, площадь, количество комнат, год постройки и тп, описание признаков можно найти по ссылке [__Ames Housing dataset__](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)).

Изучить линейную зависимость стоимости домов (SalePrice) от всех остальных показателей.

In [ ]:
data = pd.read_csv('House_prices_corrected.csv')

In [ ]:
data.head()

In [ ]:
x = data.drop(['SalePrice'], 1)
y = data['SalePrice']

В `sklearn` есть несколько классов, реализующих линейную регрессию:
* [`LinearRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) &mdash; "классическая" линейная регрессия с оптимизацией MSE
* [`Ridge`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Ridge.html) &mdash; линейная регрессия с оптимизацией MSE и $\ell_2$-регуляризацией
* [`Lasso`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.Lasso.html) &mdash; линейная регрессия с оптимизацией MSE и $\ell_1$-регуляризацией

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=10)

In [ ]:
#lr = LinearRegression()
lr = Ridge() #по умолчанию alpha=1.0, fit_intercept=True
lr.fit(x_train, y_train)

y_hat_test = lr.predict(x_test)
print('Test MSE %.3f' % mean_squared_error(y_test, y_hat_test))
print('Test R2 %.3f' % r2_score(y_test, y_hat_test))

Для сравнения:

In [ ]:
y_hat_train = lr.predict(x_train)
print("Train MSE = %.3f" % mean_squared_error(y_train, y_hat_train))
print("Train R2 = %.3f" % r2_score(y_train, y_hat_train))

### Отбор признаков

Посмотрим на то, какие признаки оказались самыми "сильными". Для этого визуализируем коэффициенты регрессии, соответствующие признакам.

In [ ]:
#Пример применения zip
a = [1,2,3]
b = [10, 20 ,30]
c = [100, 200, 300]
list(zip(a, b, c))

In [ ]:
sorted(zip(a, b, c), reverse=True) #сортировка по 1ым элементам по убыванию

In [ ]:
def show_weights(features, weights, scales):
    fig, axs = plt.subplots(figsize=(14, 10), ncols=2) #ncols=2: два графика (два столбца)
    sorted_weights = sorted(zip(weights, features, scales), reverse=True) #сортировка по весам по убыванию
    weights = [x[0] for x in sorted_weights]
    features = [x[1] for x in sorted_weights]
    scales = [x[2] for x in sorted_weights]
    sns.barplot(y=features, x=weights, ax=axs[0])
    axs[0].set_xlabel("Weight")
    sns.barplot(y=features, x=scales, ax=axs[1])
    axs[1].set_xlabel("Scale")
    plt.tight_layout()

In [ ]:
show_weights(x_train.columns, lr.coef_, x_train.std())

Будем масштабировать наши признаки. Это сделает нашу регуляризацию более честной: теперь все признаки будут регуляризоваться в равной степени. 

Для этого воспользуемся трансформером [`StandardScaler`](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html). Трансформеры в `sklearn` имеют методы `fit` и `transform` (а еще `fit_transform`). Метод `fit` принимает на вход обучающую выборку и считает по ней необходимые значения (среднее и стандартное отклонение каждого из признаков). `transform` применяет преобразование к переданной выборке.

In [ ]:
scaler = StandardScaler()
x_train_scaled = scaler.fit_transform(x_train)
x_test_scaled = scaler.transform(x_test) #на test делаем только transform!

lr = Ridge()
lr.fit(x_train_scaled, y_train)

In [ ]:
show_weights(x_train.columns, lr.coef_, x_train_scaled.std(axis=0)) #type(x_train_scaled) = numpy.ndarray

### Подбор гиперпараметров

Наряду с параметрами, которые модель оптимизирует на этапе обучения (коэффициенты регрессии), у модели есть и гиперпараметры. У нашей модели это `alpha` &mdash; коэффициент регуляризации. 

Будем пользоваться кросс-валидацией для подбора гиперпараметров.

Подберем `alpha` по логарифмической сетке, чтобы узнать оптимальный порядок величины.

In [ ]:
from sklearn.metrics import make_scorer

def r2_squared(y_true, y_pred):
    r2_coef = r2_score(y_true, y_pred)
    return r2_coef

r2_scorer = make_scorer(r2_squared, greater_is_better=True)

In [ ]:
alphas = np.logspace(-2, 3, 20)
searcher = GridSearchCV(Ridge(), [{"alpha": alphas}], scoring=r2_scorer, cv=10) #scoring="r2"
searcher.fit(x_train_scaled, y_train)

best_alpha = searcher.best_params_["alpha"]
print("Best alpha = %.4f" % best_alpha)

plt.plot(alphas, searcher.cv_results_["mean_test_score"])
plt.xscale("log")
plt.xlabel("alpha")
plt.ylabel("CV score")

In [ ]:
lr = Ridge(best_alpha) 
lr.fit(x_train_scaled, y_train)

y_hat_test = lr.predict(x_test_scaled)
print('Test MSE %.3f' % mean_squared_error(y_test, y_hat_test))
print('Test R2 %.3f' % r2_score(y_test, y_hat_test))

In [ ]:
lr.intercept_

In [ ]:
lr = Ridge() 
lr.fit(x_train_scaled, y_train)

y_hat_test = lr.predict(x_test_scaled)
print('Test MSE %.3f' % mean_squared_error(y_test, y_hat_test))
print('Test R2 %.3f' % r2_score(y_test, y_hat_test))

In [ ]:
lr.intercept_ #обратим внимание на то, что константа в регрессии не регуляризуется